In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd 

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [2]:
url = "https://bhtr.itsyourrace.com/EventCheck.aspx?id=384"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# create list of potential 2-letter combinations for searching purposes

vowels = ['a', 'e', 'i', 'o', 'u']
consonants = ['b', 'c', 'd', 'f', 'g', 'h','j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
combined_list = []

for vowel in vowels:
    for consonant in consonants:
        # combined_list.append(f"{vowel}{consonant}")
        combined_list.append(f"{consonant}{vowel}")

len(combined_list)

105

In [7]:
# create function to pull table search results

entrants_list = []
hometown_list = []
distance_list = []

def get_entrant():

    try:
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        main_area = soup.find("div", id="ctl00_ContentPlaceHolder1_pnlResults")

        entrants_table = main_area.find('table', class_="regCheck")

        entrants = entrants_table.find_all('tr')


        for i in range(1,len(entrants)):
        
            name = entrants[i].find('td', class_='col2').text.strip()
            location = entrants[i].find('td', class_='col3').text.strip()
            distance = entrants[i].find('td', class_='col1').text.strip()

            if name in entrants_list:
                next
            
            else:
                entrants_list.append(name)
                hometown_list.append(location)
                distance_list.append(distance)
    except:
        next


In [8]:

for name in combined_list:

    browser.fill('ctl00$ContentPlaceHolder1$txtLastName', name)

    browser.find_by_name('ctl00$ContentPlaceHolder1$btnSearch').first.click()
    
    get_entrant()

In [9]:
# create df from generated lists
entrants_df = pd.DataFrame()

entrants_df['entrant'] = entrants_list
entrants_df['hometown'] = hometown_list
entrants_df['distance'] = distance_list

entrants_df

,entrant,hometown,distance
0,BEEF ARCHAMBAULT,"Poplar, MT",32M
1,Ray Bader,"Casper, WY",18M
2,BEN BAEDKE,"Evansville, WY",32M
3,STEVE BAIARDI,Rowley,100M
4,AMBERLEY BAKER,"Jackson, WY",18M
...,...,...,...
1549,MADDOX MCQUEEN,"Douglas, WY",32M
1550,SARA QUINN,"Belgrade, MT",52M
1551,GREG STUART,"Thousand Oaks, CA",52M
1552,Della Stump,"Hardin, MT",32M


In [10]:
# split into state and town columns
hometown = entrants_df['hometown'].str.split(',', expand=True)[0]
homestate = entrants_df['hometown'].str.split(',', expand=True)[1]

entrants_df['hometown'] = hometown
entrants_df['homestate'] = homestate

#reorder columns, sort by state
entrants_df = entrants_df.iloc[:,[0,1,3,2]].sort_values(by='homestate')


In [11]:
entrants_df

,entrant,hometown,homestate,distance
350,STEPHEN VIOLA,Cochrane,AB,32M
737,DREW SOMMERFELDT,Lethbridge,AB,32M
397,MATTHEW MANTELLO,Calgary,AB,32M
796,KARYN HIGHET,Calgary,AB,52M
1356,JOAN NORRIS,Lethbridge,AB,18M
...,...,...,...,...
1543,BRIAN SUKUT,Livingston,None,100M
1544,CHELSEA CLUFF,Reno,None,100M
1546,JACOB LUTZ,Waterville,None,100M
1548,LAURA MC MUNN,Vancouver,None,100M


In [18]:
last_first = []
for i, row in entrants_df.iterrows():
    last_first.append(f"{entrants_df['entrant'][i].split(' ', 1)[1]}, {entrants_df['entrant'][i].split(' ', 1)[0]}")



['VIOLA, STEPHEN',
 'SOMMERFELDT, DREW',
 'MANTELLO, MATTHEW',
 'HIGHET, KARYN',
 'NORRIS, JOAN',
 'EISENREICH, BRANDON',
 'KOZUB, JON',
 'WILSON, PAT',
 'KANASHIRO, DERRICK',
 'PENNER, KEN',
 'SNOW, KIMBER',
 'PADDICK, KESHIA',
 'REYNA, EMILIO',
 'MCFARLAND, MICHELLE',
 'WALLACE, DAVID',
 'HUBAND, NATHAN',
 'MEARNS, FRANK',
 'AGUIRRE, ANDREAS',
 'LOWE, ALEX',
 'MAHRE, MICHAEL',
 'MAHRE, SEAN',
 'ZEICHICK, ANTHONY',
 'JOSHI, KAVYA',
 'KIROVSKI, TOM',
 'HERZOG, PAUL',
 'LEE, SONG',
 'SIEGEL, AMY',
 'FREEMAN, EBEN',
 'MITROS, BARBIE',
 'EVAUL, GRETCHEN',
 'GUERARD, SIMON',
 'ASMUTH, PETER',
 'STUART, GREG',
 'MCGEHEE, WILLIAM',
 'GALLIPO, MADISON',
 'BERTRAND, SANDRA',
 'VAN WILLIGEN, RONA',
 'KA, KUMMI',
 'PETERSEN, DREW',
 'AYRES, EMILY',
 'EVANS, RYAN',
 'KITTRELL, SYDNEY',
 'DVORAK, NICOLLE',
 'Hiebenthal, Laura',
 'HUNTER, TAMMY',
 'STETTNER, SEAN',
 'GRAY, LONI',
 'WESCOTT, JENNIFER',
 'BYERS, MEGAN',
 'MONSER-GRAY, KRISTIN',
 'GOLDHAMMER, GRADY',
 'HIGGINSON, CHARLOTTE',
 'MCGINTY

In [12]:
# create df for each distance
bighorn100_df = entrants_df[(entrants_df['distance'] == '100M') | (entrants_df['distance'] == '100M Wait List')]
bighorn52_df = entrants_df[(entrants_df['distance'] == '52M') | (entrants_df['distance'] == '52M Wait List')]
bighorn32_df = entrants_df[(entrants_df['distance'] == '32M') | (entrants_df['distance'] == '32M Wait List')]
bighorn18_df = entrants_df[(entrants_df['distance'] == '18M') | (entrants_df['distance'] == '18M Wait List')]

In [13]:
bighorn100_df

,entrant,hometown,homestate,distance
351,DAVID WALLACE,Phoenix,AZ,100M
511,SIMON GUERARD,Carlsbad,CA,100M
1051,DREW PETERSEN,Silverthorne,CO,100M
1488,TED BROSS,Golden,CO,100M
113,YUICHIRO HIDAKA,Kamisu,Ibaraki,100M
...,...,...,...,...
1543,BRIAN SUKUT,Livingston,None,100M
1544,CHELSEA CLUFF,Reno,None,100M
1546,JACOB LUTZ,Waterville,None,100M
1548,LAURA MC MUNN,Vancouver,None,100M


In [14]:
with pd.ExcelWriter("bighorn2024_entrants.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    bighorn100_df.to_excel(writer, sheet_name="100M", index=False)
    bighorn52_df.to_excel(writer, sheet_name="52M", index=False)
    bighorn32_df.to_excel(writer, sheet_name="32M", index=False)
    bighorn18_df.to_excel(writer, sheet_name="18M", index=False)